### Exp: 1 working with Non-snapshot

Assumption: every same vehicle in the same parking timespan have a customer that belongs to that car

Vehicle : check , same type, same cam , timestamp in the lifespan +- grace windows (padding periods) x,y is near each other ~~appear at least 2 timeframe~~ on the same x,y on non-snapshot cam

OUTPUT : vehicle_index : {timestamp range, x, y, vehicle type, }

camera used : mbk-14-12-vehicle 

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger.Approch1 import match_cross_to_vehicle, match_reverse_to_vehicle

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

CURRENT_CAM = "mbk-14-11"

df_vehicle = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-vehicle-object-20240428.csv")
)
df_cross = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-cross-object-20240428.csv")
)
df_reverse = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-reverse-object-20240428.csv")
)

print(df_vehicle[[*conf.VEHICLE_INTEREST_SNAPSHOT]].to_string())
df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
    included_vehicle_type=[
        "van",
        "bus",
        "truck",
        "NaN",
        "pickup",
        "other",
        "sedan",
        "taxi",
        "motorbike",
        "suv",
        "hatchback",
    ],
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_SNAPSHOT)

# df_vehicle[conf.VEHICLE_INTEREST_SNAPSHOT]

# drop the one that lifetime is less than 4 seconds
df_vehicle = df_vehicle.loc[(df_vehicle["lifetime"] > 4),]

# create a timestamp_unix at the end of lifespan
# df_vehicle["timestamp_unix_end"] = df_vehicle["timestamp_unix"] + (df_vehicle["lifetime"] * 1.2)
# print(df_vehicle[[*conf.VEHICLE_INTEREST_SNAPSHOT,'timestamp_unix','timestamp_unix_end']].to_string())

/home/jeans/internship/parking-customer-count
                          camera           timestamp_precise  xmin  xmax  ymin  ymax     lifetime plate_number similarized_plate_number vehicle_type  vehicle_type_confidence
0     mbk-14-12-vehicle-snapshot         2024-04-28 16:55:01  1334  2139     0   461     1.838000          NaN                      NaN          NaN                      NaN
1     mbk-14-12-vehicle-snapshot  2024-04-28 16:56:15.104000  1341  2137     0   463    37.470000          NaN                      NaN          NaN                      NaN
2     mbk-14-12-vehicle-snapshot  2024-04-28 16:57:30.083000  1343  2139     0   463    37.469000          NaN                      NaN          NaN                      NaN
3     mbk-14-12-vehicle-snapshot  2024-04-28 16:58:45.062000  1336  2137     0   463    37.469000          NaN                      NaN          NaN                      NaN
4     mbk-14-12-vehicle-snapshot  2024-04-28 16:59:59.161000  1336  2137     0   466

## Cross vs Vehicle

In [2]:
df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT,'timestamp_unix']].to_string())

## Reverse vs Vehicle

In [3]:
df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

         camera       timestamp_precise  xmin  xmax  ymin  ymax  timestamp_unix
1722  mbk-14-11 2024-04-28 02:00:15.521  1722  1925   224   519      1714269615
1720  mbk-14-11 2024-04-28 02:05:56.230  2075  2143     0   170      1714269956
1721  mbk-14-11 2024-04-28 02:06:56.733  1752  1932   107   463      1714270016
1718  mbk-14-11 2024-04-28 02:10:21.000  2036  2164    18   278      1714270221
1719  mbk-14-11 2024-04-28 02:11:19.743  1558  1745   320   615      1714270279
1711  mbk-14-11 2024-04-28 02:19:42.366  2068  2146    32   236      1714270782
1710  mbk-14-11 2024-04-28 02:21:08.546  2017  2162    32   274      1714270868
1702  mbk-14-11 2024-04-28 02:30:57.989  1434  1757   192   770      1714271457
1687  mbk-14-11 2024-04-28 02:50:44.632  1464  1663   124   772      1714272644
1683  mbk-14-11 2024-04-28 03:02:21.326  1485  1661   133   491      1714273341
1682  mbk-14-11 2024-04-28 03:04:42.287  1164  1413   332   983      1714273482
1684  mbk-14-11 2024-04-28 03:04:52.164 

## TOGETHER


In [4]:
df_vehicle = match_cross_to_vehicle(
    df_cross,
    df_vehicle,
)
df_vehicle = match_reverse_to_vehicle(df_reverse, df_vehicle)

df_vehicle = df_vehicle[(df_vehicle["timestamp_unix"] >= 1714284000) & (df_vehicle["timestamp_unix_end"] <= 1714290320)]
df_vehicle = df_vehicle[
    [
        *conf.VEHICLE_INTEREST_SNAPSHOT,
        "cross_count",
        "reverse_count",
        "original_video_name",
        "frame_no",
    ]
]

df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_SNAPSHOT)
df_vehicle

,camera,timestamp_precise,xmin,xmax,ymin,ymax,lifetime,plate_number,similarized_plate_number,vehicle_type,vehicle_type_confidence,cross_count,reverse_count,original_video_name,frame_no
1867,mbk-14-11-vehicle,2024-04-28 06:02:29.843,1110,1667,0,241,18.875,41360,41360,sedan,0.856645,2,0,mbk-14-11-vehicle_1714284001.mp4,1198
1860,mbk-14-11-vehicle,2024-04-28 06:05:35.235,1143,1670,0,252,28.751,6กม9678,6กม9678,sedan,0.936296,0,1,mbk-14-11-vehicle_1714284301.mp4,281
1861,mbk-14-11-vehicle,2024-04-28 06:06:08.106,1094,1712,0,281,15.516,มฎ5222,มฎ5222,suv,0.344409,0,0,mbk-14-11-vehicle_1714284301.mp4,544
1862,mbk-14-11-vehicle,2024-04-28 06:06:28.620,1281,1789,0,374,32.111,3กฐ7375,3กฐ7375,hatchback,0.997955,7,0,mbk-14-11-vehicle_1714284301.mp4,708
1850,mbk-14-11-vehicle,2024-04-28 06:09:36.567,1133,1700,0,358,30.955,ทส4264,ทล4264,sedan,0.969459,0,0,mbk-14-11-vehicle_1714284301.mp4,2212
1851,mbk-14-11-vehicle,2024-04-28 06:10:12.521,1393,1867,0,393,35.316,4ขภ5400,4ขก5400,pickup,0.780132,2,0,mbk-14-11-vehicle_1714284601.mp4,99
1853,mbk-14-11-vehicle,2024-04-28 06:12:27.004,1117,1733,0,348,22.594,2ขจ8645,2ขจ8645,pickup,0.861191,2,4,mbk-14-11-vehicle_1714284601.mp4,1175
1854,mbk-14-11-vehicle,2024-04-28 06:13:05.633,1082,1761,0,337,31.511,5กน9372,5กน9372,pickup,0.959077,1,0,mbk-14-11-vehicle_1714284601.mp4,1484
1855,mbk-14-11-vehicle,2024-04-28 06:14:44.965,1033,1960,0,370,64.382,NaN,NaN,van,0.260607,1,0,mbk-14-11-vehicle_1714284601.mp4,2279
1821,mbk-14-11-vehicle,2024-04-28 06:19:35.928,1133,1663,0,267,30.596,มฎ8573,มฎ8573,truck,0.929406,5,1,mbk-14-11-vehicle_1714284901.mp4,2207
